<a href="https://colab.research.google.com/github/endteamschoolofai/END/blob/main/Session9/Assignment_9_Neural_Machine_Translation_by_Jointly_Learning_to_Align_and_Translate_dataset_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 - Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.*italicized text*

## Introduction

As a reminder, here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.

## Preparing Data

Again, the preparation is similar to last time.

First we import all the required modules.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext import data 
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import pandas as pd

Define the device.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Set the random seeds for reproducability.

In [3]:
device

device(type='cuda')

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Dataset 3 - AmbigQA

In [5]:
!wget https://nlp.cs.washington.edu/ambigqa/data/ambignq.zip 
!unzip ambignq.zip

--2021-01-09 14:19:54--  https://nlp.cs.washington.edu/ambigqa/data/ambignq.zip
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18639517 (18M) [application/zip]
Saving to: ‘ambignq.zip’

ambignq.zip         100%[===================>]  17.78M  22.2MB/s    in 0.8s    

2021-01-09 14:19:55 (22.2 MB/s) - ‘ambignq.zip’ saved [18639517/18639517]

Archive:  ambignq.zip
  inflating: LICENSE                 
  inflating: train.json              
  inflating: dev.json                


In [6]:
train_df_third = pd.read_json('train.json')
test_df_third = pd.read_json('dev.json')

In [7]:
train_df_third.head()

,viewed_doc_titles,used_queries,annotations,nq_answer,id,nq_doc_title,question
0,[The Simpsons],[{'query': 'When did the simpsons first air on...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[December 17 , 1989]",-4469503464110108160,The Simpsons,When did the simpsons first air on television?
1,[John Adams (miniseries)],"[{'query': 'John adams tv', 'results': [{'titl...","[{'type': 'singleAnswer', 'answer': ['David Mo...",[David Morse],4790842463458965504,John Adams (miniseries),Who played george washington in the john adams...
2,[Marriage age in the United States],"[{'query': 'legal age of marriage in usa', 're...","[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[18, Nebraska ( 19 ), Mississippi ( 21 )]",-6631915997977101312,Age of marriage in the United States,What is the legal age of marriage in usa?
3,"[Barefoot in the Park, Barefoot in the Park (f...",[{'query': 'Who starred in barefoot in the par...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[Robert Redford, Elizabeth Ashley]",-3098213414945179648,Barefoot in the Park,Who starred in barefoot in the park on broadway?
4,"[Timeline of the Manhattan Project, Manhattan ...",[{'query': 'When did the manhattan project beg...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...",[From 1942 to 1946],-927805218867163520,Timeline of the Manhattan Project,When did the manhattan project began and end?


In [8]:
test_df_third.question , test_df_third.annotations

(0                Who plays the doctor in dexter season 1?
 1       How often does spermatogeneis—the production o...
 2          When was the first remote control tv invented?
 3         Why did the st louis cardinals move to arizona?
 4       How many episodes are in season 2 of chesapeak...
                               ...                        
 1997                How long is a rainbow six siege game?
 1998     Where did the free settlers settle in australia?
 1999        Real name of gwen stacy in amazing spiderman?
 2000     Who owns the biggest house in the united states?
 2001               Who played ben parish in the 5th wave?
 Name: question, Length: 2002, dtype: object,
 0       [{'type': 'singleAnswer', 'answer': ['Tony Gol...
 1       [{'type': 'singleAnswer', 'answer': ['usually ...
 2       [{'type': 'singleAnswer', 'answer': ['1950']},...
 3       [{'type': 'singleAnswer', 'answer': ['mediocri...
 4       [{'type': 'singleAnswer', 'answer': ['10']}, {...
          

In [9]:
train_df_third.shape, test_df_third.shape

((10036, 7), (2002, 7))

In [10]:
train_df_third = train_df_third.head(3000)
test_df_third = test_df_third.head(500)

In [11]:
train_df_third.question[1] , train_df_third.annotations[1]

('Who played george washington in the john adams series?',
 [{'answer': ['David Morse'], 'type': 'singleAnswer'}])

In [12]:
len(train_df_third.annotations[0][0]['qaPairs'])

2

In [13]:
train_df_third.annotations[0][0]['qaPairs'][0]

{'answer': ['April 19, 1987'],
 'question': 'When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?'}

In [14]:
train_df_third.question[0] , train_df_third.annotations[0][0]

('When did the simpsons first air on television?',
 {'qaPairs': [{'answer': ['April 19, 1987'],
    'question': 'When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?'},
   {'answer': ['December 17, 1989'],
    'question': 'When did the Simpsons first air as a half-hour prime time show?'}],
  'type': 'multipleQAs'})

In [15]:
train_questions_third = []
train_answers_third = []
for i in range(len(train_df_third)):
  if (train_df_third.annotations[i][0]['type'] == 'singleAnswer'):
    question = train_df_third.question[i]
    answer = train_df_third.annotations[i][0]['answer'][0]
    train_questions_third.append(question)
    train_answers_third.append(answer)
  elif (train_df_third.annotations[i][0]['type'] == 'multipleQAs'):
    for j in range(len(train_df_third.annotations[i][0]['qaPairs'])):
      question = train_df_third.annotations[i][0]['qaPairs'][j]['question']
      answer = train_df_third.annotations[i][0]['qaPairs'][j]['answer'][0]
      train_questions_third.append(question)
      train_answers_third.append(answer)


test_questions_third = []
test_answers_third = []
for i in range(len(test_df_third)):
  if (test_df_third.annotations[i][0]['type'] == 'singleAnswer'):
    question = test_df_third.question[i]
    answer = test_df_third.annotations[i][0]['answer'][0]
    test_questions_third.append(question)
    test_answers_third.append(answer)
  elif (test_df_third.annotations[i][0]['type'] == 'multipleQAs'):
    for j in range(len(test_df_third.annotations[i][0]['qaPairs'])):
      question = test_df_third.annotations[i][0]['qaPairs'][j]['question']
      answer = test_df_third.annotations[i][0]['qaPairs'][j]['answer'][0]
      test_questions_third.append(question)
      test_answers_third.append(answer)

In [16]:
train_questions_third

['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?',
 'When did the Simpsons first air as a half-hour prime time show?',
 'Who played george washington in the john adams series?',
 'What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?',
 'What is the legal age of marriage, without parental consent or other authorization, in Nebraska?',
 'What is the legal age of marriage, without parental consent or other authorization, in Mississippi?',
 'What is youngest legal age of marriage possible in some US states when circumstances permit?',
 'Who starred in barefoot in the park on broadway as Corie Bratter?',
 'Who starred in barefoot in the park on broadway as Victor Velasco?',
 'Who starred in barefoot in the park on broadway as Mrs. Banks?',
 'Who starred in barefoot in the park on broadway as Paul Bratter?',
 'Who starred in barefoot in the park on broadway as Telephone Man?',
 'Wh

In [17]:
train_answers_third

['April 19, 1987',
 'December 17, 1989',
 'David Morse',
 '18 years of age',
 '19',
 '21',
 '0',
 'Elizabeth Ashley',
 'Kurt Kasznar',
 'Mildred Natwick',
 'Robert Redford',
 'Herbert Edelman',
 'Joseph Keating',
 'Began 1939, end 1946',
 'Began 1942, end 1946',
 'June 21, 2016',
 'Aravali Range, Satpura Range, Vindhyan Range',
 '1980',
 '2009',
 '1990',
 '2005',
 '2016',
 '2018',
 '2019',
 '2014',
 'Rob Parissi',
 'Vanilla Ice',
 'Roxanne',
 'Blériot XI',
 'clear blue',
 'common humanity which is shared by all human persons.',
 'of the same being',
 'nature of God in Christianity',
 'All Shook Up',
 'Scarlett Johansson',
 'Sterling Holloway',
 'Joseph J Terry',
 'State courts',
 'October 3, 2017',
 'tied',
 'The Islands of Aloha',
 'February 1, 2017',
 'May 10, 2018',
 'May 25, 2018',
 'oil',
 'AUH',
 'OMAA',
 'Zach Johnson',
 'Sacajawea',
 'Jim Croce',
 'Hawkins sheriff Jim Hopper',
 '6',
 '6',
 '5',
 'Terry Fator',
 '2018',
 '2018',
 '2018',
 'Borinquen',
 'Traylor Howard',
 '7',
 '

In [18]:
train_df_third.shape, test_df_third.shape

((3000, 7), (500, 7))

In [19]:
len(train_questions_third), len(train_answers_third), len(test_questions_third), len(test_answers_third)

(5757, 5757, 1151, 1151)

In [20]:
Questions_Third = data.Field(sequential = True, tokenize = 'spacy', batch_first =True)#, include_lengths=True)
Answers_Third = data.Field(tokenize ='spacy', is_target=True, batch_first =True)#, sequential =False)

In [21]:
fields = [('questions_third', Questions_Third),('answers_third', Answers_Third)]

In [22]:
example1 = [data.Example.fromlist([train_questions_third[i],train_answers_third[i]], fields) for i in range(len(train_questions_third))] 
example2 = [data.Example.fromlist([test_questions_third[i],test_answers_third[i]], fields) for i in range(len(test_questions_third))] 

In [23]:
Train_Dataset_Third = data.Dataset(example1, fields)
Valid_Dataset_Third = data.Dataset(example2, fields)

In [24]:
(len(Train_Dataset_Third), len(Valid_Dataset_Third))

(5757, 1151)

In [25]:
print(vars(Train_Dataset_Third.examples[0]))
print(vars(Valid_Dataset_Third.examples[0]))

{'questions_third': ['When', 'did', 'the', 'Simpsons', 'first', 'air', 'on', 'television', 'as', 'an', 'animated', 'short', 'on', 'the', 'Tracey', 'Ullman', 'Show', '?'], 'answers_third': ['April', '19', ',', '1987']}
{'questions_third': ['Who', 'plays', 'the', 'doctor', 'in', 'dexter', 'season', '1', '?'], 'answers_third': ['Tony', 'Goldwyn']}


In [26]:
Questions_Third.build_vocab(Train_Dataset_Third, min_freq = 2)
Answers_Third.build_vocab(Train_Dataset_Third, min_freq = 2)

In [27]:
print('Size of input vocab : ', len(Questions_Third.vocab))
print('Size of label vocab : ', len(Answers_Third.vocab))
print('Top 10 words appreared repeatedly :', list(Questions_Third.vocab.freqs.most_common(10)))
print('Labels : ', Answers_Third.vocab.stoi)

Size of input vocab :  4443
Size of label vocab :  1812
Top 10 words appreared repeatedly : [('the', 6367), ('?', 5765), ('in', 3110), ('of', 2449), ('Who', 2170), ('is', 1278), ('When', 1234), ('What', 941), ('did', 918), ('was', 831)]
Labels :  defaultdict(<function _default_unk_index at 0x7f2bcc35df28>, {'<unk>': 0, '<pad>': 1, ',': 2, 'and': 3, 'the': 4, 'of': 5, '2017': 6, '-': 7, '2018': 8, 'The': 9, '"': 10, 'September': 11, 'July': 12, 'October': 13, 'June': 14, 'May': 15, 'April': 16, '1': 17, 'November': 18, 'March': 19, 'December': 20, 'August': 21, 'January': 22, '2016': 23, 'John': 24, '2': 25, '12': 26, '2015': 27, '15': 28, '6': 29, 'United': 30, '25': 31, 'New': 32, '10': 33, '4': 34, 'February': 35, 'a': 36, 'in': 37, 'to': 38, "'s": 39, '17': 40, 'Michael': 41, '13': 42, '7': 43, '9': 44, '16': 45, '3': 46, '8': 47, 'James': 48, ')': 49, '(': 50, '14': 51, '23': 52, '26': 53, '21': 54, '11': 55, '20': 56, '30': 57, '18': 58, 'States': 59, '24': 60, '.': 61, '5': 62, '

In [28]:
BATCH_SIZE = 128
train_iterator_third, valid_iterator_third = data.BucketIterator.splits(
                                (Train_Dataset_Third, Valid_Dataset_Third), 
                                batch_size = BATCH_SIZE,
                                sort_key = lambda x: len(x.questions_third),
                                sort_within_batch=True ,
                                device = device)

In [29]:
for i, batch in enumerate(train_iterator_third):
  print(type(batch), type(batch.answers_third), type(batch.questions_third), batch.answers_third.shape, batch.questions_third.shape)
  break

<class 'torchtext.data.batch.Batch'> <class 'torch.Tensor'> <class 'torch.Tensor'> torch.Size([128, 49]) torch.Size([128, 9])


## Building the Seq2Seq Model

### Encoder

First, we'll build the encoder. Similar to the previous model, we only use a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [31]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [32]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

This is the first model where we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is similar to the last two. The only difference is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [33]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [34]:
INPUT_DIM_3 = len(Questions_Third.vocab)
OUTPUT_DIM_3 = len(Answers_Third.vocab)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc3 = Encoder(INPUT_DIM_3, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec3 = Decoder(OUTPUT_DIM_3, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model3 = Seq2Seq(enc3, dec3, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [35]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            

In [36]:
model3.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4443, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1812, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=1812, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The third model has {count_parameters(model3):,} trainable parameters')

The third model has 11,283,476 trainable parameters


We create an optimizer.

In [38]:
optimizer3 = optim.Adam(model3.parameters())

We initialize the loss function.

In [39]:
TRG_PAD_IDX3 = Answers_Third.vocab.stoi[Answers_Third.pad_token]

criterion3 = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX3)

We then create the training loop...

In [40]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.questions_third
        trg = batch.answers_third

        src = src.permute(1, 0)
        trg = trg.permute(1, 0)
        optimizer.zero_grad()
        
        output = model(src, trg)

        #print("trg- ", trg.shape)
        #print("output- ", output.shape)
                
        #trg = [trg len, batch size]                                 trg-  torch.Size([4, 128])
        #output = [trg len, batch size, output dim]                  output-  torch.Size([4, 128, 3821])

        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        #print("output-", output.shape)
        trg = trg[1:]
        trg = torch.reshape(trg, (-1,)) #trg[1:].view(-1)
        #print("trg-", trg.shape)
        
        #trg = [(trg len - 1) * batch size]                             trg- torch.Size([508]) 
        #output = [(trg len - 1) * batch size, output dim]              output- torch.Size([384, 3821])
                        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [41]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            src = batch.questions_third
            trg = batch.answers_third

            src = src.permute(1, 0)
            trg = trg.permute(1, 0)
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:]#.view(-1)
            trg = torch.reshape(trg, (-1,))

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [42]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [43]:
N_EPOCHS = 10
CLIP = 1

In [44]:
best_valid_loss = float('inf')
print("Model-3")
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model3, train_iterator_third, optimizer3, criterion3, CLIP)
    valid_loss = evaluate(model3, valid_iterator_third, criterion3)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model3.state_dict(), 'tut3-model3.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Model-3
Epoch: 01 | Time: 0m 5s
	Train Loss: 5.835 | Train PPL: 341.900
	 Val. Loss: 4.132 |  Val. PPL:  62.288
Epoch: 02 | Time: 0m 5s
	Train Loss: 4.901 | Train PPL: 134.485
	 Val. Loss: 3.822 |  Val. PPL:  45.675
Epoch: 03 | Time: 0m 5s
	Train Loss: 4.441 | Train PPL:  84.899
	 Val. Loss: 3.506 |  Val. PPL:  33.309
Epoch: 04 | Time: 0m 5s
	Train Loss: 4.106 | Train PPL:  60.709
	 Val. Loss: 3.440 |  Val. PPL:  31.193
Epoch: 05 | Time: 0m 5s
	Train Loss: 3.837 | Train PPL:  46.404
	 Val. Loss: 3.370 |  Val. PPL:  29.069
Epoch: 06 | Time: 0m 5s
	Train Loss: 3.584 | Train PPL:  36.012
	 Val. Loss: 3.522 |  Val. PPL:  33.865
Epoch: 07 | Time: 0m 5s
	Train Loss: 3.346 | Train PPL:  28.395
	 Val. Loss: 3.475 |  Val. PPL:  32.307
Epoch: 08 | Time: 0m 5s
	Train Loss: 3.092 | Train PPL:  22.023
	 Val. Loss: 3.581 |  Val. PPL:  35.919
Epoch: 09 | Time: 0m 5s
	Train Loss: 2.863 | Train PPL:  17.522
	 Val. Loss: 3.780 |  Val. PPL:  43.803
Epoch: 10 | Time: 0m 5s
	Train Loss: 2.581 | Train PPL: 